### Aqueous Organic Estimator

Authors: Grayson Boyer, Jordyn Robare

A notebook for estimating the thermodynamic properties and HKF parameters of aqueous organic molecules using second order group additivity methods.

Date modified: 10/30/2020

In [ ]:
input_name = "alkene_test_mini.csv"

In [ ]:
from IPython.display import SVG
from rdkit import Chem
from rdkit.Chem import rdDepictor, Draw
from rdkit.Chem.Draw import rdMolDraw2D
import pandas as pd
import math
import sigfig
import pubchempy as pcp
import os
import thermo
from chemparse import parse_formula

# for benson group additivity
from pgradd.GroupAdd.Library import GroupLibrary
import pgradd.ThermoChem

In [ ]:
# properties of the elements
# Cox, J. D., Wagman, D. D., and Medvedev, V. A., CODATA Key Values for Thermodynamics, Hemisphere Publishing Corp., New York, 1989.
# Compiled into a CSV by Jeffrey Dick for CHNOSZ

element_data = pd.read_csv("data/element.csv", index_col="element")
element_data = element_data.loc[element_data['source'] == "CWM89"]

# 
def entropy(name, unit="J/mol/K"):
    """ Calculate the standard molal entropy of elements in a molecule.
    Note: requires a dataframe called element_data to be in global namespace from:
    
    ```element_data = pd.read_csv("data/element.csv", index_col="element")
    element_data = element_data.loc[element_data['source'] == "CWM89"]```
    
    The file 'element.csv' was formatted by Jeffrey Dick for the CHNOSZ package for R.
    """
    this_compound = pcp.get_compounds(name, "name")
    formula = parse_formula(this_compound[0].molecular_formula)
    entropies = [(element_data.loc[elem, "s"]/element_data.loc[elem, "n"])*formula[elem] for elem in list(formula.keys())]
    if unit == "J/mol/K":
        unit_conv = 4.184
    elif unit == "cal/mol/K":
        unit_conv = 1
    else:
        print("Error in entropy: specified unit", unit, "is not recognized. Returning entropy in J/mol/K")
        unit_conv = 4.184
        
    return sum(entropies)*unit_conv

entropy("hexane")

In [ ]:
# convert a formula dictionary into a formula string
def dict_to_formula(formula_dict):
    formula_string = ""
    for key in formula_dict.keys():
        if abs(formula_dict[key]) == 1:
            v = ""
        else:
            v = formula_dict[key]
            if (v).is_integer():
                v = int(v)

        formula_string = formula_string + str(key) + str(v)
    return formula_string

dict_to_formula(parse_formula("CO3-2"))

In [ ]:
# load functions

def match_group(name, pattern):
    this_compound = pcp.get_compounds(name, "name")
    this_smile = this_compound[0].canonical_smiles
    mol = Chem.MolFromSmiles(this_smile)
    functional_group = Chem.MolFromSmarts(pattern)
    matches = mol.GetSubstructMatches(functional_group)
    return len(matches)

def match_groups(name, patterns, draw=False):
    this_compound = pcp.get_compounds(name, "name")
    this_smile = this_compound[0].canonical_smiles
    mol = Chem.MolFromSmiles(this_smile) #convert smiles from internet to usable form for rdkit
    pattern_matches = [len(mol.GetSubstructMatches(Chem.MolFromSmarts(pattern))) for pattern in patterns] #!!!
    # figure out how higher order groups were estimated and see if we can convert to
    # 2nd order groups - see if that makes any sense
    
    match_dict = dict(zip(patterns, pattern_matches))
    
    ### check that total formula of groups matches that of the molecule
    
    # create a dictionary of element matches
    total_formula_dict = {}
    for match in match_dict.keys():
        this_match = parse_formula(pattern_dict[match])
        for element in this_match.keys():
            this_match[element] *= match_dict[match]
            if element in total_formula_dict:
                total_formula_dict[element] += this_match[element]
            else:
                total_formula_dict[str(element)] = 0
                total_formula_dict[element] += this_match[element]
    
    # remove keys of elements with a value of 0 (e.g. "H":0.0)
    for key in list(total_formula_dict.keys()):
        if total_formula_dict[key] == 0.0:
            total_formula_dict.pop(key, None)
    
    # retrieve individual charges that contribute to net charge
    atomic_info = this_compound[0].record["atoms"]
    chargedict = {}
    if "charge" in atomic_info.keys():
        all_charges = [chargedict.get("value", 0) for chargedict in atomic_info["charge"]]
        pos_charge = sum([charge for charge in all_charges if charge > 0])
        neg_charge = abs(sum([charge for charge in all_charges if charge < 0]))
        if pos_charge > 0:
            chargedict['+']=float(pos_charge)
        if neg_charge > 0:
            chargedict['-']=float(neg_charge)
    else:
        chargedict = {}

    # perform the comparison
    test_dict = parse_formula(this_compound[0].molecular_formula)
    test_dict.update(chargedict)
    if total_formula_dict != test_dict:
        print("Warning! The formula of", name, "does not equal the the elemental composition of the matched groups! This could be because the structure of", name, "is missing representative groups.")
        print("Formula of", name +":")
        pcp_dict = parse_formula(this_compound[0].molecular_formula)
        pcp_dict.update(chargedict)
        print(pcp_dict)
        print("Total formula of group matches:")
        print(total_formula_dict)
    
    # add molecular formula to match dictionary
    match_dict["formula"] = dict_to_formula(total_formula_dict)
    
    ### create a png and svg of the molecule
    if draw:
        molSize=(450,150)
        mc = Chem.Mol(mol.ToBinary())
        if not mc.GetNumConformers():
            #Compute 2D coordinates
            rdDepictor.Compute2DCoords(mc)
        # init the drawer with the size
        drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
        #draw the molcule
        drawer.DrawMolecule(mc)
        drawer.FinishDrawing()
        # get the SVG string
        svg = drawer.GetDrawingText()
        # fix the svg string and display it
    #     display(SVG(svg.replace('svg:','')))
        os.makedirs("mol_svg",exist_ok=True)
        os.makedirs("mol_png",exist_ok=True)
        #Draw.MolToFile( mol, "mol_svg/"+name+".svg" )
        Draw.MolToFile( mol, "mol_png/"+name+".png" )
    return match_dict


def print_group_matches(name, patterns): #unused function
    this_compound = pcp.get_compounds(name, "name")
    this_smile = this_compound[0].canonical_smiles
    mol = Chem.MolFromSmiles(this_smile)
    for pattern in patterns:
        functional_group = Chem.MolFromSmarts(pattern)
        matches = mol.GetSubstructMatches(functional_group)
        if(len(matches) != 0):
            print(len(matches), pattern)

def display_molecule(name, save=False):
    this_compound = pcp.get_compounds(name, "name")
    this_smile = this_compound[0].canonical_smiles
    mol = Chem.MolFromSmiles(this_smile)
    molSize=(450,150)
    mc = Chem.Mol(mol.ToBinary())
    if not mc.GetNumConformers():
        #Compute 2D coordinates
        rdDepictor.Compute2DCoords(mc)
    # init the drawer with the size
    drawer = rdMolDraw2D.MolDraw2DSVG(molSize[0],molSize[1])
    #draw the molcule
    drawer.DrawMolecule(mc)
    drawer.FinishDrawing()
    # get the SVG string
    svg = drawer.GetDrawingText()
    # fix the svg string and display it
    display(SVG(svg.replace('svg:','')))
    if save:
        os.makedirs("mol_svg",exist_ok=True)
        os.makedirs("mol_png",exist_ok=True)
        #Draw.MolToFile( mol, "mol_svg/"+name+".svg" )
        Draw.MolToFile( mol, "mol_png/"+name+".png" )

def get_smarts(name):
    this_compound = pcp.get_compounds(name, "name")
    this_smile = this_compound[0].canonical_smiles
    return Chem.MolToSmarts(Chem.MolFromSmiles(this_smile))

def get_smiles(name):
    this_compound = pcp.get_compounds(name, "name")
    this_smile = this_compound[0].canonical_smiles
    return Chem.MolFromSmiles(this_smile)

In [ ]:
# functions for benson group additivity

def BensonG(name, print_groups=False):
    this_compound = pcp.get_compounds(name, "name")
    this_smile = this_compound[0].canonical_smiles
    lib = GroupLibrary.Load('BensonGA')
    descriptors = lib.GetDescriptors(this_smile)
    if print_groups:
        print(descriptors)
    thermochem = lib.Estimate(descriptors,'thermochem')
    return thermochem.get_G(298.15, units="kJ/mol")

def BensonHSCp(name, print_groups=False):
    this_compound = pcp.get_compounds(name, "name")
    this_smile = this_compound[0].canonical_smiles
    lib = GroupLibrary.Load('BensonGA')
    descriptors = lib.GetDescriptors(this_smile)
    if print_groups:
        print(descriptors)
    thermochem = lib.Estimate(descriptors,'thermochem')
    return thermochem.get_H(298.15, units="kJ/mol"), thermochem.get_S(298.15, units="J/mol/K"), thermochem.get_Cp(298.15, units="J/mol/K")

molecule = "ethanol"
H, S, Cp = BensonHSCp(molecule)
print("Calculated from H and S:", H - 298.15*(S - entropy(molecule))/1000)
print("Calculated from get_G:", BensonG(molecule))
print("get_Cp:", Cp)

In [ ]:
#check if thermo package calls on more complex databases than the original

# molecule = "2-methoxypropane" #secondary ether
#molecule = "2-methylhexanenitrile" #secondary ester
molecule = "1-butanol" #secondary ester

mol = get_smiles(molecule)
display_molecule(molecule)
J = thermo.Joback(mol) 
mol_Gf = J.estimate()['Gf']/1000 # ideal gas Joback estimation of the Gibbs free energy of formation
print("Ideal Gas Gf =", round(mol_Gf, 2), "kJ/mol")

In [ ]:
pattern_dict = {
    
    #corrections suggested in literature
    "[O](-C[CH3])-C[CH2]": "", #verified with 1-ethoxybutane, correction for ethoxyalkanes
    "O-[CH2]-[CH2]-O": "", #verified with 1,2-diethoxyethane, correction for di- and polyethers     
    "[CH2](-[CH2]-C#N)-C#N": "", #verified for butanedinitrile, for correction
    "[CH2](-[CH2]-[CH2]-C#N)-C#N": "", #verified for pentanedinitrile, for correction    
  
    #our corrections based on higher order group contributions in papers
    "N#C-[CH](-C)-C": "", #verified for 2-methylhexanenitrile, use for correction
    "N#C-[CH2]-C": "", #verified for hexanenitrile, use for correction
    "C-[CH2]-C=O": "", #verified with octanoic acid, use for correction
    "C-[CH](-C)-C=O": "", #2-methyloctanoic acid, use for correction
    "C-[C](-C)(-C)-C=O": "", #2,2-dimethyloctanoic acid, use for correction
    "C-[C](=O)-O-C": "", #ethyl propanoate, provisional ester group, correction on top of other groups: [O]=C, C-[O]-C
    "[CH](=O)-OC": "", #isopropyl formate, provisional methanoate group, correction on top of other groups: [O]=C, C-[O]-C
    "O=C-O-[CX4H1](-C)-C":"", #secondary ester, isopropyl ethanoate, correction on top of secondary ether
    "C-O-[CX4H0](-C)(-C)-C":"", #tertiary ester, tert-butyl ethanoate, correction on top of tertiary ether
    
    #can't match ethyne, ethylene oxide,   
    
    #imines
    "C-[C](-C)=N": "C", #verified with hexafluoroacetone imine TypeError: unsupported operand type(s) for +=: 'float' and 'str'
    "C=[NH]" : "NH", #verified with hexafluoroacetone imine
    
   #alkenes
    "C=[CX2H0]=C": "C", #verified with propadiene
    "C=[CX3H2]": "CH2", #verified with propadiene ERROR
    "C=[CX3H1]-C": "CH", #verified with propene
    "C=[CX3H0](-C)-C": "C", #verified with methyl cyanoacrylate ERROR
    
    #alkynes
    "C#[CX2H1]": "CH", #verified with propyne ERROR
    "C-[CX2H0]#C": "C", #verified with 2-butyne ERROR
    
    #tropylium
    "c[cX3H1+]c": "CH+", #verified with tropylium
    
    #pyrrole
    "n[cX3H1]c": "CH", #verified with pyrrole ERROR
    "c[nX3H1]c": "NH", #verified with pyrrole ERROR
    
    #cyano/nitrile groups
    "C#[NX1H0]": "N", #verified with cyanobenzene ERROR
    "C-[CX2H0]#N": "C", #2-methylhexanenitrile
    
    #carbon monoxide
    "O#[CX1H0-]": "C-", #verified with carbon monoxide
    "C#[OX1H0+]": "O+", #verified with carbon monoxide
   
    #nitro groups
    "N-[O-]": "O-", #verified with nitrobenzene ERROR
    "N=[O]": "O",  #verified with nitrobenzene ERROR
    
    #halogens
    "C-[Br]": "Br", #verified with 2-bromobutanamide ERROR
    "C-[CX4H2]-Br": "CH2", #verified with 2-bromoethanamide
    "C-[CX4H1](-C)-Br": "CH", #verified with 2-bromobutanamide
    "C-[C](-C)(-C)-Br": "C",
    "C-[C](-Br)(-Br)-Br": "C", #verified with hexabromoacetone
    "C-[C](-Br)=O": "C", #verified with acetyl bromide
    
    "C-[Cl]": "Cl", 
    "C-[CX4H2]-Cl": "CH2", 
    "C-[CX4H1](-C)-Cl": "CH",
    "C-[C](-C)(-C)-Cl": "C",
    "C-[C](-Cl)(-Cl)-Cl": "C", 
    "C-[C](-Cl)=O": "C",

    "C-[F]": "F", 
    "C-[CX4H2]-F": "CH2", 
    "C-[CX4H1](-C)-F": "CH",
    "C-[C](-C)(-C)-F": "C",
    "C-[C](-F)(-F)-F": "C",
    "C-[C](-F)=O": "C",

    "C-[I]": "I", 
    "C-[CX4H2]-I": "CH2", 
    "C-[CX4H1](-C)-I": "CH",
    "C-[C](-C)(-C)-I": "C",
    "C-[C](-I)(-I)-I": "C", 
    "C-[C](-I)=O": "C",
    
    # aliphatic carbon
    "C-[CX4H3]":"CH3", #verified for hexane
    "C-[CX4H2]-C":"CH2", #verified for hexane
    "C-[CX4H1](-C)-C":"CH", #verified for 2-methylpropane
    "C-[CX4H0](-C)(-C)-C":"C1", #verified for 2,2-dimethylpropane
    
    # primary, secondary, tertiary alcohols, ethers, and esters
    "O-[CX4H3]":"CH3", #verified for methoxyethane
    "O-[CX4H2]-C":"CH2", #verified for methoxyethane
    "C-[CX4H1](-C)[OH]":"CH", #isopropanol, secondary alcohol
    "C-[CX4H0](-C)(-C)[OH]":"C", #1,1-dimethylethanol, tertiary alcohol
    "C-[CX4H1](-O-C)-C":"CH", #verified for 2-methoxypropane, secondary ether
    "C-O-[CX4H0](-C)(-C)-C":"C", #verified for 2-methoxy-2-methylpropane, tertiary ether
    "C-[OX2H]":"OH", #verified for methanol
    "C-[O-]": "O-", #verified with formate
    "C-[OX2]-C": "O", #verified for methoxyethane
    "O-[CH2]-O": "CH2",
    "O-[CH](-C)-O": "CH",
    
    #primary amines
    "C-[CX4H2]-N": "CH2", #verified with ethanolamine ERROR
    "C-[NX3H2]": "NH2", #verified with ethanolamine, will not longer match with dimethylammonium
    "C-[NH3+]": "NH3+", #verified with ethylammonium
    
    #things connected to nitrogens
    "N-[CX4H3]": "CH3", #verified with dimethylamine ERROR
    "N-[CX4H1](-C)-C": "CH1", #verified with 1-methylethylamine ERROR
    "N-[CX4H0](-C)(-C)-C": "C", #verified with 1,1-dimethylethylamine ERROR
    
    #secondary amines
    "C-[NX3]-C": "NH1", #verified with dimethyLamine ERROR
    "C-[NH2+]-C": "NH2+", #verified with dimethylammonium ERROR
    
    #tertiary amines
    "C-[NX3](-C)-C": "N", #verified for trimethylamine ERROR
    "C-[NH1+](-C)-C": "NH+", #verified for trimethylammonium
    
    #quaternary amines
    "C-[N+](-C)(-C)-C": "N+", #verified for tetramethylammonium
   
    #thiols
    "C-[SX2H1]": "SH", #verified with methanethiol ERROR
    "C-[S-]": "S-", #verified with methanethiolate
    "C-[SX2]-C": "S",
    "S-[CH3]": "CH3", #verified with methanethiol
    "S-[CH2]-C": "CH2", 
    "S-[CH](-C)-C": "CH", #verified with 3-pentanethiol ERROR
    "S-[C](-C)(-C)-C": "C", 
    "S-[c](c)c": "C", #verified with thiophenol ERROR
    "c-[SH]": "SH", #verified with thiophenol 
    "c-[S-]": "S-", 
    "c[s]c": "S", #verified with thiophene ERROR
    "s[c]c": "C", #verified with thiophene
    "c-[S]-S": "S", #verified with diphenyl disulfide ERROR
    "S-[c](c)c": "C", #verified with diphenyl disulfide
    "S-[S]-C": "S", 
    "S-[S]-S": "S",
    
    #carbonyls
    "C=[O]": "O", #verified with formaldehyde
    "O=[CX3H2]": "CH2", #verified with formaldehyde
    "O=[CX3H1]-O": "CH", #verified with formic acid - group can't be separated Plyasunov et al., 2004a
    "O=[CX3](-O)-O": "C", #verified with carbonic acid
    "O=[CX3](-C)-O": "C", #verified with acetic acid- group can't be separated Plyasunov et al., 2004a
    "O=[CX3](-O)-c": "C", #verified with benzoic acid
    "O=[CX3H1]-N": "CH", #verified with methanamide ERROR 
    "O=[CX3](-C)-N": "C", #verified with ethanamide ERROR
    "O=[CX3](-c)-N": "C", #verified with benzamide ERROR
    "O=[CX3H1]-c": "CH", #verified with benzaldehyde
    "C-[C](-C)=O": "C", #verified with hexabromoacetone
    
    #sulfur carbonyls
    "C=[S]": "S", #verified with thiobenzophenone #ERROR
    "S=[CX3H2]": "CH2", #verified with thioformaldehyde
    "S=[CX3H1]-O": "CH", #verified with
    "S=[CX3](-O)-O": "C", #verified with
    "S=[CX3](-C)-O": "C", #verified with
    "S=[CX3](-O)-c": "C", #verified with
    "S=[CX3H1]-N": "CH", #verified with 
    "S=[CX3](-C)-N": "C", #verified with
    "S=[CX3](-c)-N": "C", #verified with
    "S=[CX3H1]-c": "CH", #verified with 
    "C-[C](-C)=S": "C", #verified with 
    
     #aromatic
    "c[cX3H+0]c": "CH", #verified with benzene, 
    "c[cX3H0](c)c": "C", #verified with napthalene
    "c-[CX4H2]-c": "CH2", #verified with fluorene
    
    #aromatic branching
    "c-[CX2H0]#N": "C", ##verified with cyanobenzene ERROR
    "c-[NX3H0+](=O)-O": "N+", #verified with nitrobenzene ERROR
    "c-[NX3H2]": "NH2", #verified with aniline ERROR
    "c[cX3](-N)c": "C", #verified with methylbenzene
    "c-[CX4H3]": "CH3", #verified with methylbenzene
    "c[cX3](-C)c": "C", #verified with methylbenzene
    "c-[CX4H2]-C": "CH2", #verified with ethylbenzene
    "c-[CX4H1](-C)-C": "CH", #verified with isopropylbenzene
    "c-[CX4H0](-C)(-C)-C": "C", #verified with tert-butylbenzene
    "c-[CX3H0](=O)-c": "C", #verified with benzophenone
    "c-[CX3H0](=S)-c": "C", #verified with sulfobenzophenone
    
    #aromatic nitrogen
    #esters
}

In [ ]:
# test group matching and error messages
try:
    test_result = match_groups(molecule, pattern_dict.keys(), draw=False)
except:
    print("test failed... maybe the molecule name is spelled incorrectly?")

test_result

In [ ]:
# load alcohol test csv
df_inp = pd.read_csv("data/"+input_name)
df_inp.head()

In [ ]:
# get list of molecules to look up
molecules = df_inp["compound"]
molecules.head()

In [ ]:
# create a df of names and groups
df = pd.DataFrame()
vetted_mol = []
for molecule in molecules:
    try:
        df = df.append(match_groups(molecule, pattern_dict.keys(), draw=True), ignore_index=True)
        vetted_mol.append(molecule) #matches online
    except:
        pass
df.index = vetted_mol

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
props = df_inp[[colname for colname in df_inp.columns.values if colname not in ["compound"]]]
prop_names = df_inp.columns.values[df_inp.columns.values != 'compound']
props.index = molecules
props.head()

In [ ]:
df_c = pd.concat([props, df], axis=1, sort=False)
df_c.to_csv("data/organic_second_order_alc_test.csv", index_label="compound")
df_c.head()

In [ ]:
# remove columns with no matches
df_c = df_c.loc[:, (df_c.sum(axis=0) != 0)]
df_c.head()

In [ ]:
# load compiled 2nd order group contribution (gc) data
df_gc = pd.read_csv("data/group_contribution_data.csv", dtype=str)
df_gc.head()

In [ ]:
# set index to smarts code
df_gc = df_gc.set_index("smarts")
df_gc.head()

In [ ]:
# get a list of relevent groups
groups = [group for group in df_c.columns if group not in ["formula"]+list(prop_names)]
groups

In [ ]:
def findHKF(Gh=float('NaN'), Vh=float('NaN'), Cp=float('NaN'),
            Gf=float('NaN'), Hf=float('NaN'), Saq=float('NaN'),
            charge=float('NaN'), J_to_cal=True):

    # define eta (angstroms*cal/mol)
    eta = (1.66027*10**5)

    # define YBorn (1/K)
    YBorn = -5.81*10**-5

    # define QBorn (1/bar)
    QBorn = 5.90*10**-7

    # define XBorn (1/K^2)
    XBorn = -3.09*10**-7

    # define abs_protonBorn (cal/mol), mentioned in text after Eq 47 in Shock and Helgeson 1988
    abs_protonBorn = (0.5387 * 10**5)


    if not pd.isnull(Gh) and charge == 0:

        # find omega*10^-5 (j/mol) if neutral and Gh available
        HKFomega = 2.61+(324.1/(Gh-90.6)) # Eq 8 in Plyasunov and Shock 2001

    elif charge == 0:

        # find omega*10^-5 (j/mol) if neutral and Gh unavailable
        HKFomega = (10^-5)*((-1514.4*(Saq/4.184)) + (0.34*10**5))*4.184 # Eq 61 in Shock and Helgeson 1990 for NONVOLATILE neutral organic species

    elif charge != 0:

        # define alphaZ (described in text after Eq 59 in Shock and Helgeson 1990)
        if (abs(charge) == 1):
            alphaZ = 72
        elif (abs(charge) == 2):
            alphaZ = 141
        elif (abs(charge) == 3):
            alphaZ = 211
        elif (abs(charge) == 4):
            alphaZ = 286
        else:
            alphaZ = float('NaN')

        # define BZ
        BZ = ((-alphaZ*eta)/(YBorn*eta - 100)) - charge*abs_protonBorn # Eq 55 in Shock and Helgeson 1990

        # find ion omega*10^-5, (J/mol) if charged
        HKFomega = (10^-5)*(-1514.4*(Saq/4.184) + BZ)*4.184 # Eq 58 in Shock and Helgeson 1990

        ### METHOD FOR INORGANIC AQUEOUS ELECTROLYTES USING SHOCK AND HELGESON 1988:

            # find rej (angstroms), ions only
            #rej <- ((charge^2)*(eta*YBorn-100))/((Saq/4.184)-71.5*abs(charge)) # Eqs 46+56+57 in Shock and Helgeson 1988

            # find ion absolute omega*10^-5, (cal/mol)
            #HKFomega_abs_ion <- (eta*(charge^2))/rej # Eq 45 in Shock and Helgeson 1988

            # find ion omega*10^-5, (J/mol)
            #HKFomega2 <- (10^-5)*(HKFomega_abs_ion-(charge*abs_protonBorn))*4.184 # Eq 47 in Shock and Helgeson 1988

    else:
        HKFomega = float('NaN')


    # find delta V solvation (cm3/mol)
    V_solv = -(HKFomega/10**-5)*QBorn*10 # Eq 5 in Shock and Helgeson 1988, along with a conversion of 10 cm3 = 1 joule/bar

    # find delta V nonsolvation (cm3/mol)
    V_nonsolv = Vh - V_solv # Eq 4 in Shock and Helgeson 1988

    # find sigma (cm3/mol)
    HKFsigma = 1.11*V_nonsolv + 1.8 # Eq 87 in Shock and Helgeson

    # find delta cp solvation (J/mol*K)
    cp_solv = ((HKFomega/10**-5)*298.15*XBorn) # Eq 35 in Shock and Helgeson 1988 dCpsolv = omega*T*X

    # find delta cp nonsolvation (J/mol*K)
    cp_nonsolv = Cp - cp_solv # Eq 29 in Shock and Helgeson 1988



    if not pd.isnull(Gh) and charge == 0:
        # find a1*10 (j/mol*bar)
        HKFa1 = (0.820-((1.858*10**-3)*(Gh)))*Vh # Eq 10 in Plyasunov and Shock 2001
        # why is this different than Eq 16 in Sverjensky et al 2014? Regardless, results seem to be very close using this eq vs. Eq 16.

        # find a2*10^-2 (j/mol)
        HKFa2 = (0.648+((0.00481)*(Gh)))*Vh # Eq 11 in Plyasunov and Shock 2001

        # find a4*10^-4 (j*K/mol)
        HKFa4 = 8.10-(0.746*HKFa2)+(0.219*Gh) # Eq 12 in Plyasunov and Shock 2001

    elif charge != 0:
        # find a1*10 (j/mol*bar)
        HKFa1 = (0.1942*V_nonsolv + 1.52)*4.184 # Eq 16 in Sverjensky et al 2014, after Plyasunov and Shock 2001, converted to J/mol*bar. This equation is used in the DEW model since it works for charged and noncharged species up to 60kb

        # find a2*10^-2 (j/mol)
        HKFa2 = (10**-2)*(((HKFsigma/41.84) - ((HKFa1/10)/4.184))/(1/(2601)))*4.184 # Eq 8 in Shock and Helgeson, rearranged to solve for a2*10^-2. Sigma is divided by 41.84 due to the conversion of 41.84 cm3 = cal/bar

        # find a4*10^-4 (j*K/mol)
        HKFa4 = (10**-4)*(-4.134*(HKFa2/4.184)-27790)*4.184 # Eq 88 in Shock and Helgeson, solve for a4*10^-4

    else:
        HKFa1 = float('NaN')
        HKFa2 = float('NaN')
        HKFa3 = float('NaN')

    # find c2*10^-4 (j*K/mol)
    if not pd.isnull(Gh) and charge == 0:
        HKFc2 = 21.4+(0.849*Gh) # Eq 14 in Plyasunov and Shock 2001
    elif not pd.isnull(Cp) and charge != 0:
        HKFc2 = (0.2037*(Cp/4.184) - 3.0346)*4.184 # Eq 89 in Shock and Helgeson 1988
    else:
        HKFc2 = float('NaN')


    # find c1 (j/mol*K)
    HKFc1 = cp_nonsolv-(((HKFc2)/10**-4)*(1/(298.15-228))**2) # Eq 31 in Shock and Helgeson 1988, rearranged to solve for c1

    # find a3 (j*K/mol*bar)
    HKFa3 = (((Vh/10)-(HKFa1/10)-((HKFa2/10**-2)/2601)+((HKFomega/10**-5)*QBorn))/(1/(298.15-228)))-((HKFa4/10**-4)/2601) # Eq 11 in Shock and Helgeson 1988, rearranged to solve for a3. Vh is divided by 10 due to the conversion of 10 cm3 = J/bar

    if J_to_cal:
        conv = 4.184
    else:
        conv = 1
    
    # report results in calorie scale, ready to be pasted into OBIGT
    out = {
      "G" : (Gf/conv)*1000,
      "H" : (Hf/conv)*1000,
      "S" : Saq/conv,
      "Cp" : Cp/conv,
      "V" : Vh,
      "a1" : HKFa1/conv,
      "a2" : HKFa2/conv,
      "a3" : HKFa3/conv,
      "a4" : HKFa4/conv,
      "c1" : HKFc1/conv,
      "c2" : HKFc2/conv,
      "omega" : HKFomega/conv,
      "Z" : charge,
      "Vsolv" : V_solv,
      "Vnonsolv" : V_nonsolv,
      "sigma" : HKFsigma}

    return out

# findHKF(Gh=-80.74,       Vh=68.16, Cp=105, Gf=5.795, Hf=-129.0, Saq=76.6, charge=-1) # phenolate
# findHKF(Gh=float('NaN'), Vh=68.16, Cp=105, Gf=5.795, Hf=-129.0, Saq=76.6, charge=-1) # phenolate 1988 method

# findHKF(Vh=-25.4, Cp=-1.3*4.184, Gf=(-83500*4.184)/1000, Hf=(-91500*4.184)/1000, Saq=-55.7*4.184, charge=2) # Be+2
# findHKF(Vh=18.13, Cp=15.74*4.184, Gf=(-18990*4.184)/1000, Hf=(-31850*4.184)/1000, Saq=26.57*4.184, charge=1) # NH4+
# findHKF(Vh=-0.87, Cp=14.2*4.184, Gf=(-69933*4.184)/1000, Hf=(-66552*4.184)/1000, Saq=2.70*4.184, charge=1) # Li+

# Compare to table 4 of Plyasunov and Shock 2001
# (may be slightly different due to using Eq 16 in Sverjensky et al 2014 for calculating a1)
findHKF(Gh=-0.51, Vh=39.0, Cp=146, charge=0, J_to_cal=False) # SO2
# findHKF(Gh=-11.7, Vh=77.1, Cp=306, charge=0, J_to_cal=False) # Pyridine
# findHKF(Gh=-37.7, Vh=88.23, Cp=347, charge=0, J_to_cal=False) # 1,4-Butanediol
# findHKF(Gh=-74, Vh=58.7, Cp=76, charge=0, J_to_cal=False) # beta-alanine


In [ ]:
# function to find number of significant digits. Requires a string.
# modified from a solution from https://stackoverflow.com/questions/8142676/
def find_sigfigs(x):
    '''Returns the number of significant digits in a number. This takes into account
       strings formatted in 1.23e+3 format and even strings such as 123.450'''
    # change all the 'E' to 'e'
    x = x.lower()
    if ('-' == x[0]):
        x = x[1:]
    if ('e' in x):
        # return the length of the numbers before the 'e'
        myStr = x.split('e')
        return len( myStr[0] ) - 1 # to compenstate for the decimal point
    else:
        # put it in e format and return the result of that
        ### NOTE: because of the 8 below, it may do crazy things when it parses 9 sigfigs
        n = ('%.*e' %(8, float(x))).split('e')
        # remove and count the number of removed user added zeroes. (these are sig figs)
        if '.' in x:
            s = x.replace('.', '')
            #number of zeroes to add back in
            l = len(s) - len(s.rstrip('0'))
            #strip off the python added zeroes and add back in the ones the user added
            n[0] = n[0].rstrip('0') + ''.join(['0' for num in range(l)])
        else:
            #the user had no trailing zeroes so just strip them all
            n[0] = n[0].rstrip('0')
        #pass it back to the beginning to be parsed
    return find_sigfigs('e'.join(n))

find_sigfigs("5.220")

In [ ]:
# a function to round to n sig figs
# solution from https://stackoverflow.com/questions/3410976
import math
round_to_n = lambda x, n: x if x == 0 else round(x, -int(math.floor(math.log10(abs(x)))) + (n - 1))
round_to_n(55.2, 2)

In [ ]:
df_c.head()

In [ ]:
# check whether property names include "Gh", "Hh", "Sh", "Cph", and "V"
# The HKF estimator and OBIGT generator cannot proceed unless the dataframe has these properties.

props = ["Gh", "Hh", "Sh", "Cph", "V"]

try:
    assert all(prop in props for prop in list(prop_names))
except:
    print('Error: cannot proceed with property estimation unless molecules have the following properties: "Gh", "Hh", "Sh", "Cph", and "V"')
    print('Supplied properties:', list(prop_names))
    assert False

In [ ]:
# create a dataframe to store estimated properties of molecules
df_est = pd.DataFrame(index=df_c.index)

hkf_params = ["a1", "a2", "a3", "a4", "c1", "c2", "omega"]

ig_method = "Joback" # Group contribution method for ideal gas properties. "Joback" or "Benson".

for prop in props:
    # add column for property and error
    df_est[prop] = ""
    df_est[prop+"_err"] = ""   

for param in hkf_params:
    # add column for parameter
    df_est[param] = ""

for prop in ["Gig", "Hig", "Sig", "Cpig", "Gaq", "Haq", "Saq", "Cpaq"]:
    df_est[param] = ""
    
df_est["note"] = ""
    
for molecule in df_c.index:
    
    formula = df_c.loc[molecule, "formula"]
    
    for prop in props:
        
        #print('\nPROPERTY:', prop, "\n")
        err_str = prop + "_err"
        
        # derive Sh, entropy of hydration, in J/mol K
        if prop == "Sh":
            try:
                # S = (G-H)/(-Tref)
                mol_prop = (float(df_est.loc[molecule, "Gh"]) - float(df_est.loc[molecule, "Hh"]))/(-298.15)
                mol_prop = mol_prop*1000 # convert kJ/molK to J/molK
                
                # propagate error from Gh and Hh to estimate Sh error.
                # equation used: Sh_err = Sh*sqrt((Gh_err/Gh)^2 + (Hh_err/Hh)^2)
                Gh_err = float(df_est.loc[molecule, "Gh_err"])/float(df_est.loc[molecule, "Gh"])
                Hh_err = float(df_est.loc[molecule, "Hh_err"])/float(df_est.loc[molecule, "Hh"])
                mol_err = abs(mol_prop)*math.sqrt(Gh_err**2 + Hh_err**2)
                
                # check whether Gh or Hh as the fewest sigfigs
                sf = min([find_sigfigs(df_est.loc[molecule, p]) for p in ["Gh", "Hh"]])
                
                # round Sh to this number of sigfigs
                mol_prop = sigfig.round(str(mol_prop), sigfigs=sf)
                
                # check how many decimal places Sh has after sigfig rounding
                if "." in mol_prop:
                    this_split = mol_prop.split(".")
                    n_dec = len(this_split[len(this_split)-1])
                else:
                    n_dec = 0
                
                # assign Sh and Sh_err to dataframe
                df_est.at[molecule, prop] = mol_prop
                df_est.at[molecule, err_str] = format(mol_err, '.'+str(n_dec)+'f')
                
            except:
                pass
            
            continue
            
        else:
            pass
        
        # initialize variables and lists
        mol_prop = 0
        mol_err = 999
        prop_errs = []
        n_dec = 999
        error_groups = []

        for group in groups:
            
            try:
                contains_group = df_c.loc[molecule, group][0] != 0
            except:
                contains_group = df_c.loc[molecule, group] != 0

            # if this molecule contains this group...
            if contains_group:

                try:
                    # add number of groups multiplied by its contribution
                    mol_prop += df_c.loc[molecule, group] * float(df_gc.loc[group, prop])

                    # round property to smallest number of decimal places
                    if "." in df_gc.loc[group, prop]:
                        this_split = df_gc.loc[group, prop].split(".")
                        n_dec_group = len(this_split[len(this_split)-1])
                    else:
                        n_dec_group = 0
                        
                    if n_dec_group < n_dec:
                        n_dec = n_dec_group
                        
                    # handle group std errors
                    try:
                        float(df_gc.loc[group, err_str]) # assert that this group's error is numeric
                        prop_errs.append(df_gc.loc[group, err_str]) # append error
                    except:
                        # if group's error is non-numeric, pass
                        pass

                except:
                    error_groups.append(group)

        if len(error_groups) == 0:

            # add Y0
            mol_prop += float(df_gc.loc["Yo", prop])

            # propagate error of summed groups: sqrt(a**2 + b**2 + ...)
            mol_err = round(math.sqrt(sum([float(err)**2 for err in prop_errs])), n_dec) # propagate error from groups
            
            # format output
            mol_prop = format(mol_prop, '.'+str(n_dec)+'f')
            mol_err = format(mol_err, '.'+str(n_dec)+'f')
            
            #print(molecule, "\t\t", mol_prop, u'\u00b1', mol_err)
            df_est.at[molecule, prop] = mol_prop
            df_est.at[molecule, err_str] = mol_err

        else:
            message1 = molecule + " encountered errors with group(s): " + str(error_groups) + "."
            message2 = "Are these groups assigned properties in the data file? ;"
            df_est.at[molecule, "note"] = df_est.loc[molecule, "note"] + message1 + " " + message2
            print(message1)
            print(message2)
    
    try:
        Selements = entropy(molecule)
    except:
        print("Error! Could not calculate entropy from the elements of", molecule)
        Selements = float("NaN")
    
    if ig_method == "Joback":
        # Joback estimation of the Gibbs free energy of formation of the ideal gas
        try:
            mol = get_smiles(molecule)
            J = thermo.Joback(mol) 
            Gig = J.estimate()['Gf']/1000
            Hig = J.estimate()['Hf']/1000
            Sig = ((Gig - Hig)/-298.15)*1000 + Selements
            Cpig = J.estimate()['Cpig'](T=298.15)
        except:
            Gig = float("NaN")
            Hig = float("NaN")
            Sig = float("NaN")
            Cpig = float("NaN")
    
    elif ig_method == "Benson":
        # Benson estimation of the Gibbs free energy of formation of the ideal gas
        try:
            Hig, Sig, Cpig = BensonHSCp(molecule)
            delta_Sig = Sig - Selements
            Gig = Hig - 298.15*delta_Sig/1000
        except:
            Gig = float("NaN")
            Hig = float("NaN")
            Sig = float("NaN")
            Cpig = float("NaN")
    else:
        print("Error! The ideal gas property estimation method", ig_method, "is not recognized. Try 'Joback' or 'Benson'.")
    
    # estimate the Gibbs free energy of formation of the aqueous molecule by summing
    # its ideal gas and hydration properties.
    try:
        Gaq = Gig + float(df_est.loc[molecule, "Gh"])
        Haq = Hig + float(df_est.loc[molecule, "Hh"])
        Saq = ((Gaq - Haq)/-298.15)*1000 + Selements
        Cpaq = Cpig + float(df_est.loc[molecule, "Cph"])
        
    except:
        Gaq = float("NaN")
        Haq = float("NaN")
        Saq = float("NaN")
        Cpaq = float("NaN")
        
    df_est.at[molecule, "Gig"] = Gig
    df_est.at[molecule, "Hig"] = Hig
    df_est.at[molecule, "Sig"] = Sig
    df_est.at[molecule, "Cpig"] = Cpig
    df_est.at[molecule, "Gaq"] = Gaq
    df_est.at[molecule, "Haq"] = Haq
    df_est.at[molecule, "Saq"] = Saq
    df_est.at[molecule, "Cpaq"] = Cpaq
    df_est.at[molecule, "formula"] = formula

        
    # calculate HKF parameters
    try:
        hkf_dict = findHKF(Gh=float(df_est.loc[molecule, "Gh"]),
                           Vh=float(df_est.loc[molecule, "V"]),
                           Cp=Cpaq, Gf=Gaq, Hf=Haq,
                           Saq=Saq, charge=0, J_to_cal=False)
        for param in hkf_params:
            df_est.at[molecule, param] = hkf_dict[param]
    except:
        print("Could not calculate HKF parameters for", molecule)
        pass
        
df_est.head()

In [ ]:
df_est

In [ ]:
df_est.to_csv("raw_output.csv")

In [ ]:
# write aqueous output to OBIGT csv file
name = list(df_est.index)
abbrv = list(df_est["formula"])
formula = list(df_est["formula"])
state = ["aq"]*len(df_est.index)
ref1 = ["test"]*len(df_est.index)
ref2 = ["test"]*len(df_est.index)
date = ["test"]*len(df_est.index)
E_units = ["J"]*len(df_est.index)
G = [float(value)*1000 for value in list(df_est["Gaq"])]
H = [float(value)*1000 for value in list(df_est["Haq"])]
S = list(df_est["Saq"])
Cp = list(df_est["Cpaq"])
V = list(df_est["Vh"])
a1 = list(df_est["a1"])
a2 = list(df_est["a2"])
a3 = list(df_est["a3"])
a4 = list(df_est["a4"])
c1 = list(df_est["c1"])
c2 = list(df_est["c2"])
omega = list(df_est["omega"])
Z = ["0"]*len(df_est.index)

obigt_out = pd.DataFrame(zip(name, abbrv, formula,
                             state, ref1, ref2,
                             date, E_units, G,
                             H, S, Cp, V, a1, a2,
                             a3, a4, c1, c2,
                             omega, Z),
                         columns =['name', 'abbrv', 'formula',
                                   'state', 'ref1', 'ref2',
                                   'date', 'E_units', 'G',
                                   'H', 'S', 'Cp', 'V', 'a1.a', 'a2.b',
                                   'a3.c', 'a4.d', 'c1.e', 'c2.f',
                                   'omega.lambda', 'z.T'])


obigt_out = obigt_out.dropna() # remove any rows with 'NaN'

obigt_out.head()

In [ ]:
obigt_out.to_csv("new_alc_estimates.csv", index=False)

In [ ]:
# TODO: df_est into 'OBIGT_data0'-style file